## Índice




1.  **Instancias** 
2.  **Query + Filtrado**
3.  **Guardado**
    



### 1.0 Librerias

In [1]:
import geopandas as gpd
from pymongo import MongoClient
from shapely.geometry import box,Polygon, Point
import datetime
import csv

### 1.1 Instanciamos el mapa y la NUT de estudio

Mapas proporcionados por la Unión europea :https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts

Primero instanciamos una bounding box para pedirle al servidor que os devuelva todos los tuits que se hayan hecho en esa localización.

In [2]:
NUT_ESTUDIO='DE929'                                             #NUT arbitraria de estudio
EU_RG= gpd.read_file('mapas\\NUTS_RG_10M_2016_4326.shp')

Polygon_gpd= EU_RG[EU_RG['NUTS_ID']==NUT_ESTUDIO]['geometry']   #Aquí identificamos la NUT 
Polygon_sha = Polygon_gpd.iloc[0]                               #Tomamos la clase Polygon de Shapely

minx, miny, maxx, maxy= Polygon_sha.bounds                      #Tomamos máximos y mínimos


Bbox_json= gpd.GeoSeries([box(minx, miny, maxx, maxy)]).to_json()
bbox_dict= eval(Bbox_json)
bbox_coords= bbox_dict['features'][0]['geometry']['coordinates']  

#Esto úlimo sé que es mucho, pero es para pasar de shp a json

### 1.2 Instanciamos los elementos temporales

Para poder limpiar datos vamos a determinar un año de estudio, luego queremos observar como se van desplazando las personas y para ello primero debemos definir su residencia, y posteriormente sus viajes o mudanzas. Se definirá como residencia aquella NUT con mallor número de tweets de un usuario en un mes. 

Para filtrar bots, y fútiles, vamos a definir unos máximos y mínimos de tweets. 

In [3]:
#Instanciamos la ventana de tiempo: yy,mm,dd
beg_month= datetime.date(2019,1,1)
fin_month= datetime.date(2019,12,1)


#Ventana de tiempo de referencia
days_before= 30

beg_hist_month= beg_month - datetime.timedelta(days=days_before) 
fin_hist_month= fin_month - datetime.timedelta(days=days_before) 

#Filtros:
max_tweet_in_coll= 1000   #Máximo de tweets registrados en una misma collection
max_total_tweets= 1000   #Máximo de tweets totales
min_total_tweets= 50     #Cantidad mínima de tweets totales por usuairio

### 2. Query + Filtrado

Los datos están localizados en diferentes collections, así que deberemos registrarlas una por una.

In [2]:
uri = "Direcciión del servidor"
client = MongoClient(host=uri)

#Hacemos la conexión correctamente con with
with client:
    
    db = client['twitter'] #Modificación del cliente
    uids = set()     #Aquí meteremos todos los ID de los usuarios que han tuiteado en la NUT_ESTUDIO
    home_dict=dict() #Aquí almacenaremos el número de usuarios turistas por NUT. EJ: {'DE929':30,'DE732':15 }
    
    
    #-----------------Usuarios en la NUT_ESTUDIO en todas las coll---------------------
    
    for coll in db.list_collections(): #Debido a que la información está en más de una coll
        '''
        Aquí pedimos las coordenadas no nulas, que estén dentro de nuestro bbox, que estén en la ventana temporal estipulada.
        En cuanto al ID retiramos el del servidor y solicitamos el del usuario.
        
        '''
        doc_geo = coll.find({'coordinates' : {'$ne':  None},
                             'coordinates': {'$geoWithin':{'$geometry': {'type' : 'Polygon','coordinates': bbox_coords}}},
                             'created_at':{'$gte':beg_month,'$lt': fin_month}}, 
                            {'_id':0,'user.id':1})
        
        #Retiramos las repeticiones en general
        for user in doc_geo:
            uid = user['user']['id']
            if uid not in uids:  
                uids.add(uid)
                
    #Con esto obtenemos la lista de usuarios en la NUT de estudio de todas las coll

    #------------------------------Selección de turistas------------------------------

    for user in uids:
        
        NUT_user_dict= dict() #Aquí meteremos los lugares donde ha tuiteado un usuario con la misma estructura que home_dict.
        
        #Buscamos los tweets en todas las coll
        for coll in db.list_collections():
            
            u_rec = coll.find({'user.id': user,
                               'created_at':{'$gte':beg_hist_month,'$lt': fin_hist_month}},
                              {'_id':0,'coordinates.coordinates': 1}) #'created_at': 1 
            
            #Filtrado_01
            num_doc= u_rec.count()

            if  num_doc > max_tweet_in_coll: 
                continue

                
            #Recopilamos un 'histograma' con las veces que se repite la NUT
            for tweet in u_rec:

                coordinates= tweet['coordinates']['coordinates'][0]  #Esto es por el formato json
                tweet_point= Point(coordinates[0],coordinates[1])

                NUT_point= EU_NUT[EU_NUT.contains(tweet_point)].iloc[0]  #Esto es por como se estructuran los datos del mapa
                                                                         #primero saliendo los NUT-3

                if NUT_point not in NUT_user_dict:

                    NUT_user_dict[NUT_point]= 1

                else:
                    NUT_user_dict[NUT_point]+= 1
                
                
        #Filtro_02, ahora que tenemos TODOS los tweets de usuario podemos discriminar y tratar los datos
        
        NUT_u_keys= list(NUT_user_dict.keys())
        NUT_u_values= list(NUT_user_dict.values())
        
        total_tweets= sum(NUT_u_values)
        
        if total_tweets< min_total_tweets or total_tweets> max_total_tweets: 
            continue
        
        if max(NUT_u_values)/total_tweets < 0.7: #pedimos que el 70% de los tweets del mes anterior estén en la misma NUT
            continue

        #---------------------Evaluamos su home-----------------------

        home= NUT_u_keys[NUT_u_values.index(max(NUT_u_values))]

        if home != NUT_ESTUDIO:  #Solo guardamos turistas

            if home not in home_dict:

                home_dict[home]= 1

            else:
                home_dict[home]+= 1
                

Tras todo el proceso nos quedamos con un diccionario de todos los turistas que han pasado por la región de estudio. Sabiendo de donde vienen y el número, pero se ha perdido la identidad de cada uno.

### 3. Guardado

In [ ]:
    
with open('turistas_in_mallorca.csv','w') as file:

    fieldnames = ['NUT', 'TOURIST_NUMBER']

    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()

    home_key= list(home_dict.keys())
    home_value= list(home_dict.values())

    for i in range(len(home_key)):

        writer.writerow({'NUT': home_key[i], 'NUMBER' : home_value[i]})
        